In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gillespy2
from scipy.optimize import minimize

In [2]:
#Repressor System
def CLE(beta1, k1, n1, alpha, dt):
    x = [x0]
    for i in range(len(t) - 1):
        dxi = np.random.normal(0, 1, 3) 
        x_curr = x[-1]
        
        

        F = (beta1 * (pow(k1, n1) )/ (pow(k1, n1) + pow(x_curr, n1)) -
            alpha * x_curr)
        
        G = (
            np.sqrt(beta1 * (pow(k1, n1) / (pow(k1, n1) + pow(x_curr, n1)))) * dxi[0] +
            
            np.sqrt(alpha * x_curr) * dxi[1])
        
        x_next = x_curr + dt * F + np.sqrt(dt) * G

        x = np.hstack([x, x_next])
    return x

In [34]:
beta1=10
k1=500
n1=1
alpha=0.02
t = np.linspace(10000,100000,10000)
dt = t[1] - t[0]
x0=200
x_cle = CLE( beta1,k1, n1, alpha, dt)
x_cle_trimmed = x_cle[1000:]



In [35]:
np.mean(x_cle_trimmed)

309.76639613838546

In [36]:
np.std(x_cle_trimmed)

15.554132602772949

In [37]:
def f_rep(x_cle_trimmed, k1, n1, beta1):
    return beta1 * (pow(k1, n1) / (pow(k1, n1) + pow(x_cle_trimmed, n1)))

def f_act(x_cle_trimmed, n2, k2, beta2):
    return beta2 * (pow(x_cle_trimmed, n2) / (pow(k2, n2) + pow(x_cle_trimmed, n2)))

def f_deg(x, alpha):
    return alpha * x_cle_trimmed

In [38]:


def fixed_point_rep(x_cle_trimmed, beta1, k1, n1, alpha):
    try:
        fix_rep_pt = (-alpha * k1 + np.sqrt((k1**2) * (alpha**2) + 4 * alpha * k1*beta1)) / (2 * alpha)
        return fix_rep_pt
    except ValueError as e:
        print(f"Error: {e}")
        return None

fix_rep_pt = fixed_point_rep(x_cle_trimmed, beta1, k1, n1, alpha)
if fix_rep_pt is not None:
    print('fix_rep:', fix_rep_pt)


fix_rep: 309.01699437494744


In [39]:
A=(beta1 * n1 * pow(k1, n1) * pow(fix_rep_pt, n1 - 1))/((pow(fix_rep_pt, n1) + pow(k1, n1))**2) +alpha
B=(beta1 * pow(k1, n1))/(pow(fix_rep_pt, n1) + pow(k1, n1))+alpha*fix_rep_pt
   

In [40]:
np.sqrt(B/(2*A))

14.953487812212206

In [41]:
def fixed_point_act(x_cle_trimmed, beta2, k2, n2, alpha):
    try:
        fix_act_pt = (beta2 - alpha * k2) / alpha
        return fix_act_pt
    except ValueError as e:
        print(f"Error: {e}")
        return None


In [42]:
def A_r(fix_act_pt, beta2, n2, K2, alpha):
    denominator = pow(fix_act_pt, n2) + pow(K2, n2)
    numerator = -beta2 * n2 * pow(K2, n2) * pow(fix_act_pt, n2 - 1)
    result = (numerator / pow(denominator, 2)) + alpha
    return result

def B_r(fix_act_pt, beta2, n2, K2, alpha):
    denominator = pow(fix_act_pt, n2) + pow(K2, n2)
    numerator = beta2 * pow(fix_act_pt, n2)
    result = (numerator / denominator) + alpha * fix_act_pt
    return result


In [48]:
def cost_function(params, fix_rep_pt, x_cle_trimmed, k1, beta1, alpha, n2):
    beta2, K2 = params
    
    fix_act_pt = fixed_point_act(x_cle_trimmed, beta2, K2, n2, alpha)
    
    A = (beta1 * n1 * pow(k1, n1) * pow(fix_rep_pt, n1 - 1)) / ((pow(fix_rep_pt, n1) + pow(k1, n1))**2) + alpha
    B = (beta1 * pow(k1, n1)) / (pow(fix_rep_pt, n1) + pow(k1, n1)) + alpha * fix_rep_pt
   
    A_r_val = A_r(fix_act_pt, beta2, n2, K2, alpha)
    B_r_val = B_r(fix_act_pt, beta2, n2, K2, alpha)
    
    expression_diff = np.abs(B / (2 * A) - B_r_val / (2 * A_r_val))**2 + 10* np.abs(fix_act_pt - fix_rep_pt)**2
    
    return np.sum(expression_diff)


In [56]:
# Set up the bounds
bounds_definition = [(0, None), (0, None)] 
n2 = 1

initial_guess = [0.1, 1]

result = minimize(cost_function, initial_guess, args=(fix_rep_pt, x_cle_trimmed, k1, beta1, alpha, n1),
                  method='trust-constr', tol=1e-6, options={'maxiter': 1000},
                  bounds=bounds_definition)

optimized_params = result.x

print("Optimized parameters (beta2, K2):", optimized_params)



Optimized parameters (beta2, K2): [6.02504859 0.00000021]


In [57]:
optimized_params = [6.02504859 ,0.00000021]


fix_act_pt_optimized = fixed_point_act(x_cle_trimmed, optimized_params[0], optimized_params[1], n2, alpha)
print("Fixed point value at optimized parameters:", fix_act_pt_optimized)

Fixed point value at optimized parameters: 301.25242928999995


In [58]:
A_r_val = A_r(fix_act_pt_optimized ,optimized_params[0], n2, optimized_params[1], alpha)
print(A_r_val)
B_r_val = B_r(fix_act_pt_optimized ,optimized_params[0], n2, optimized_params[1], alpha)
print(B_r_val)
print(B_r_val/(2*A_r_val))

0.019999999986058205
12.050097171600001
301.2524295


In [59]:
print(np.sqrt(B_r_val/(2*A_r_val)))#repressor variance 

17.356624945535927
